# Auto Generated Agent Chat: Task Solving with Provided Tools as Functions (PowerShell Scripts)


In [ ]:
import autogen
import os
import subprocess

scriptlet_map = {
    'Manage_Processes': {
        'description': 'Managing processes with process cmdlets',
        'script': 'Get-Process | Format-Table -AutoSize'  # Example scriptlet to list all processes
    },
    'Manage_Services': {
        'description': 'Managing services',
        'script': 'Get-Service | Where-Object {$_.Status -eq "Running"}'  # Example scriptlet to list all running services
    },
    'Manage_VM': {
        'description': 'Managing virtual machines',
        'script': 'Get-VM'  # Example scriptlet to list all VMs
    },
    'Manage_Storage': {
        'description': 'Managing storage and file services',
        'script': 'Get-Volume'  # Example scriptlet to list all volumes
    },
    'Manage_Network': {
        'description': 'Implement and manage an on-premises and hybrid networking infrastructure',
        'script': 'Get-NetIPAddress'  # Example scriptlet to list all IP addresses
    },
    'Manage_AD_DS': {
        'description': 'Deploy and manage Active Directory Domain Services (AD DS)',
        'script': 'Get-ADUser -Filter *'  # Example scriptlet to list all AD DS users
    },
    'Windows_Update': {
        'description': 'Manage Windows Updates',
        'script': 'Install-Module -Name PSWindowsUpdate; Get-WindowsUpdate'  # Using PSWindowsUpdate module to manage Windows Updates
    },
    'Manage_Firewall': {
        'description': 'Automating Windows Defender Firewall management',
        'script': 'Get-NetFirewallProfile'  # Example scriptlet to get firewall profiles
    },
    # ... other scriptlets based on administrative tasks
}


# Define a function to execute PowerShell scriptlets
def exec_ps(script_name, *args):
    script_path = scriptlet_map.get(script_name)
    if not script_path:
        raise ValueError(f"No scriptlet found for {script_name}")
    with open(script_path, 'r') as script_file:
        script = script_file.read().format(*args)
    process = subprocess.Popen(["powershell", script], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    if error:
        return str(error), False
    return str(output), True

# Create a configuration for the AssistantAgent
llm_config = {
    "functions": [
        {
            "name": "exec_ps",
            "description": "Execute a PowerShell scriptlet and return the output.",
            "parameters": {
                "type": "object",
                "properties": {
                    "script_name": {
                        "type": "string",
                        "description": "The name of the scriptlet to be executed.",
                    },
                    "args": {
                        "type": "array",
                        "description": "Arguments to be passed to the scriptlet.",
                    },
                },
                "required": ["script_name"],
            },
        },
    ],
}

# Initialize the AssistantAgent and UserProxyAgent
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="Only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    function_map={"exec_ps": exec_ps},
)

# Start the conversation with a systems administration task
user_proxy.initiate_chat(
    chatbot,
    message="Task: List all user accounts on the system.",
)

# The conversation will proceed with the chatbot suggesting a scriptlet to execute,
# the UserProxyAgent executing the scriptlet, and communicating the result back to the chatbot.
